# Chess opening project

## Section 1: Installing modules

The below code iniatlises the required modules.

In [83]:
!pip install pandas
!pip install tabulate
import pandas as pd
!pip install numpy
import numpy as np
import traceback
pd.__version__
pd.set_option('future.no_silent_downcasting', True)

This is old code used to write games to a text file.

In [ ]:
with open("FirstLinesOfGames.txt","w") as wfile:
    with open("PGN_Database_1.pgn","r") as rfile:
        for i in range(0,20000):
            wfile.write(rfile.readline())

## Section 2: Defining functions

The below code defines the functions **ImportGames** and **ImportAllGames**. 
These functions import chess games from a local PGN file and input them into a pandas DataFrame which will be names Game_db. The function **ImportAllGames** takes the parameter number of opening moves, which decides how many opening moves of the game to list in Game_db, and **ImportGames** additionally takes a parameter for how many games to be imported from the file.

In [51]:
GameData = {}
Event = []
Site = []
Date = []
Round = []
White = []
Black = []
Result = []
WhiteElo = []
BlackElo = []
ECO = []
EventDate = []
Opening = []
WhiteFideID = []
BlackFideID = []
#Game = []
#Game_db = pd.DataFrame()

i = 0

def ImportGames(NoGames, NoOpeningMoves: int = 7, StartingFromGame = 0):  #Creates files of games and df -- if run twice will overwrite file/start from the beginning
                                                                 #StartingFromGame not yet implemented
    GameData = {} #initiates variables
    Result = []
    WhiteElo = []
    BlackElo = []
    Opening = []
    Game = []
    
    NoOpeningMovesString = str(NoOpeningMoves + 1) + "." #converts to string so we can use it to search for substring
    counter = 0

    try: 
        with open("PGN_Database_1.pgn","r", encoding="utf-8") as rfile: #opens the file of all games
            for line in rfile: #significantly faster then using readlines()
                if counter <= NoGames: #breaks if not
                    if "Result" in line: #Adds line to correct list
                        while len(Result) != counter: #Makes sure all lists are same length, while in case missed multiple in a row
                            if len(Result) < counter: 
                                Result.append("Null") #Puts null if Result data missing from game
                            else:
                                Result.pop() #Due to counter off games, if game missing then removes Result data as well
                        Result.append(line[9:-3]) #Adds only text interested to into list
            
                    elif "WhiteElo" in line:
                        while len(WhiteElo) != counter:
                            if len(WhiteElo) < counter:
                                WhiteElo.append(0) #Sets unknow ELO to 0
                            else:
                                WhiteElo.pop()
                        WhiteElo.append(line[11:-3])
        
                    elif "BlackElo" in line:
                        while len(BlackElo) != counter:
                            if len(BlackElo) < counter:
                                BlackElo.append(0)
                            else:
                                BlackElo.pop()
                        BlackElo.append(line[11:-3])
        
                    elif "Opening" in line:
                        while len(Opening) != counter:
                            if len(Opening) < counter:
                                Opening.append("Null")
                            else:
                                Opening.pop()
                        Opening.append(line[10:-3])
        
                    elif line[0:2] == "1." and "12" in line: #Game doesn't have specific word, need the dot (.) as sometimes just result e.g. 1-0 is listed, also min of 12 moves
                        counter = counter + 1
                        j = line.find(NoOpeningMovesString) #Only add the number of moves specified when function called
                        Game.append(line[:j-1])
                else:
                    break
                
            for List in (Result,Opening, Game): #Final Check if last one is missing - adding null
                while len(List) != counter:                       
                    if len(List) < counter:
                        List.append("Null")
                    else:
                        List.pop()
                        
            for List in (WhiteElo, BlackElo): #Final Check if last one is missing - adding 0
                while len(List) != counter:                          
                    if len(List) < counter:
                        List.append(0)
                    else:
                        List.pop()
        
            GameData = {"Opening":Opening,"WhiteElo":WhiteElo,"BlackElo":BlackElo,"Result":Result,"Moves":Game}
            global Game_db
            Game_db = pd.DataFrame(GameData) #creates our df and makes it global
            

        return Game_db
        
    except Exception as error:
        print("Ran into error ", Exception, error)
        print(traceback.format_exc())


def ImportAllGames(NoOpeningMoves: int = 7, StartingFromGame = 0):  #Same as ImportGames, just goes to end of file
    GameData = {}
    Result = []
    WhiteElo = []
    BlackElo = []
    Opening = []
    Game = []
    
    NoOpeningMovesString = str(NoOpeningMoves + 1) + "."
    counter = 0

    try: 
        with open("PGN_Database_1.pgn","r", encoding="utf-8") as rfile:
            for line in rfile:
                if "Result" in line: #need result to not be null - remove all
                    while len(Result) != counter:
                        if len(Result) < counter:
                            Result.append("Null")
                        else:
                            Result.pop()
                    Result.append(line[9:-3])
                    
                elif "WhiteElo" in line: #multiple missing in a row?
                    while len(WhiteElo) != counter:
                        if len(WhiteElo) < counter:
                            WhiteElo.append(0)
                        else:
                            WhiteElo.pop()
                    WhiteElo.append(line[11:-3])
        
                elif "BlackElo" in line:
                    while len(BlackElo) != counter:
                        if len(BlackElo) < counter:
                            BlackElo.append(0)
                        else:
                            BlackElo.pop()
                    BlackElo.append(line[11:-3])
        
                elif "Opening" in line:
                    while len(Opening) != counter:
                        if len(Opening) < counter:
                            Opening.append("Null")
                        else:
                            Opening.pop()
                    Opening.append(line[10:-3])
        
                elif line[0:2] == "1." and "12" in line: #COuld also put in each of Result,BlackElo etc. Think if Game empty.                   
                    counter = counter + 1
                    j = line.find(NoOpeningMovesString)
                    Game.append(line[:j-1])

                elif line == "": #Unnecessary?
                    print("Reached end of file")
                    break

            for List in (Result,Opening, Game): #Final Check if last one is missing - adding null
                while len(List) != counter:                       
                    if len(List) < counter:
                        List.append("Null")
                    else:
                        List.pop()
                        
            for List in (WhiteElo, BlackElo): #Final Check if last one is missing - adding 0
                while len(List) != counter:                          
                    if len(List) < counter:
                        List.append(0)
                    else:
                        List.pop()
        
            GameData = {"Opening":Opening,"WhiteElo":WhiteElo,"BlackElo":BlackElo,"Result":Result,"Moves":Game}
            global Game_db
            Game_db = pd.DataFrame(GameData)            

        return Game_db
        
    except Exception as error:
        print("Ran into error ", Exception, error)
        print(traceback.format_exc())


            
        

Next we define two more functions **CleanDf** and **SplitDf** below. 

The function **CleanDf** cleans the given database by removing entries that don't have a result, and converting the string result values into floats off 1, 0.5, 0 for white win, draw and black win respectively. It also sets the type of each column in the dataframe to make manipulating data easier in the future. Finally **CleanDf** takes parameters of the dataframe to be cleaned and a rating gap, which eliminates all games that have a difference in Elo of the two players higher than the given rating gap. This is to ensure the games considered for analysis will be between players of relatively close ability.

The function **SplitDf** takes our dataframe and a rating increment parameter, and then creates new dataframes of games in a specific rating range, for instance if the rating increment is 100 (which is the current default) the game database will be split into dataframes: rating[0], rating[1200], rating[1300], rating[1400] ... rating[2800], rating[2900], rating[3000]. These dataframes can then be analysed to see how the openings vary at different ratings.

In [72]:
#Game_db_new = pd.DataFrame()
#ImportAllGames(10)

def CleanDf(df,Rating_gap = 100):
    global Game_db_new
    
    df = df[df["Result"].isin(["1-0","1/2-1/2","0-1"])] #Removes any games with different result
    df.loc[:,"Result"] = df["Result"].replace({"1-0":1, "1/2-1/2":0.5,"0-1":0}) #Converts result to numbers
    df = df.astype({"Opening":"string","BlackElo":int,"WhiteElo":int,"Result":float,"Moves":"string"}) #typecasts each column "WhiteElo":int
    df = df[abs(df["WhiteElo"] - df["BlackElo"]) < Rating_gap] #Shows only games within a certain rating
    

    Game_db_new = df
    return Game_db_new

def SplitDf(Rating_increment, df = pd.DataFrame()):

    df_copy = df
    global Rating_df
    Rating_df = {}
    i = 1200
    global Rating_cieling
    Rating_cieling = [0]
    while i in range(1200,3000):
        Rating_cieling.append(i)
        i = i + Rating_increment
    print(Rating_cieling)

    j = 0
    try:
        df_copy.insert(3, "AverageElo", (df["WhiteElo"] + df["BlackElo"])/2,allow_duplicates=False)
    except Exception as error:
        print(f"Error is :{Exception}, and {error}")
    #print(df)
    while j < len(Rating_cieling)-1:

        Rating_df[Rating_cieling[j]] = df_copy[df_copy["AverageElo"].isin(range(Rating_cieling[j],Rating_cieling[j+1]))]
        j = j + 1
    #print(df)
    return
    


## Section 3: Analysing games (WIP)

In [73]:
#ImportGames(4000000)
#CleanDf(Game_db)
#print(Game_db_new.dtypes)
#Game_db_new
#ImportAllGames()
#CleanDf(Game_db)
SplitDf(100,Game_db_new)
Rating_cieling
#Game_db[Game_db.isnull()==True]


[0, 1200, 1300, 1400, 1500, 1600, 1700, 1800, 1900, 2000, 2100, 2200, 2300, 2400, 2500, 2600, 2700, 2800, 2900]
Error is :<class 'Exception'>, and cannot insert AverageElo, already exists


[0,
 1200,
 1300,
 1400,
 1500,
 1600,
 1700,
 1800,
 1900,
 2000,
 2100,
 2200,
 2300,
 2400,
 2500,
 2600,
 2700,
 2800,
 2900]

In [76]:
for rating in Rating_cieling[:-1]:
    print(f"The size of db {rating} is {Rating_df[rating].shape}")
#Game_db_new

The size of db 0 is (637377, 6)
The size of db 1200 is (403, 6)
The size of db 1300 is (632, 6)
The size of db 1400 is (884, 6)
The size of db 1500 is (1589, 6)
The size of db 1600 is (2799, 6)
The size of db 1700 is (4459, 6)
The size of db 1800 is (7488, 6)
The size of db 1900 is (14106, 6)
The size of db 2000 is (38799, 6)
The size of db 2100 is (71373, 6)
The size of db 2200 is (122712, 6)
The size of db 2300 is (145222, 6)
The size of db 2400 is (152940, 6)
The size of db 2500 is (103477, 6)
The size of db 2600 is (52501, 6)
The size of db 2700 is (18279, 6)
The size of db 2800 is (1101, 6)


In [91]:
Rating_df[2000]["Opening"].value_counts()
#Game_db.value_counts()

Opening
Null                                              19674
Sicilian                                           3596
French                                             1410
King's Indian                                       926
Queen's pawn game                                   903
                                                  ...  
Caro-Masi defence                                     1
Neo-Gruenfeld, 5.cd                                   1
King's knight's gambit                                1
Blumenfeld counter-gambit, Spielmann variation        1
Semi-Benoni (`blockade variation')                    1
Name: count, Length: 220, dtype: Int64